In [ ]:
from airtable import Airtable
import os
import pandas as pd
import ipywidgets as widgets

In [ ]:
api_key = input('Please enter the API key')
base_key = input('Please enter the base ID')

In [ ]:
table_name = 'ATLAS - Raw Data'

airtable = Airtable(base_key, table_name, api_key = api_key)

df = pd.DataFrame.from_records(r['fields'] for r in airtable.get_all())

In [ ]:
metrics = ['Overall Sustainability Score', 'Edible Portion', 'Mercury Content', 'Foodservice menu prominence -2019']
output = widgets.Output()

sust_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Sustainability')
welf_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Animal Welfare')
health_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Public Health')
market_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Market Size')

s_widgets = [sust_slider, welf_slider, health_slider, market_slider]

def handler(change):
    output.clear_output()
    raw_weights = [s_widget.value for s_widget in s_widgets]
    sum_of_weights = sum(raw_weights)
    weights = [raw_weight / sum_of_weights for raw_weight in raw_weights]
    
    # This is completely the wrong formula - this version is just a proof of concept
    df['Score'] = pd.DataFrame([pd.to_numeric(df[metric]) * weight for weight, metric in zip(weights, metrics)]).sum(axis = 0)
    df.sort_values('Score', inplace = True, ascending = False)
    
    with output:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df[['Archetype', 'Score', 'Overall Sustainability Score', 'Edible Portion', 'Mercury Content', 'Foodservice menu prominence -2019']])
    
for s_widget in s_widgets:
    s_widget.observe(handler, names = 'value')

In [ ]:
for s_widget in s_widgets:
    display(s_widget)
    
display(output)
